In [ ]:
from pathlib import Path
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define dataset YAML path
DATA = "/content/drive/MyDrive/data/object_detection_dataset/data.yaml"

Mounted at /content/drive


In [ ]:
!pip -q install ultralytics

from ultralytics import YOLO

# Load the pretrained YOLOv8-Medium model as the starting point for transfer learning
model = YOLO("yolov8m.pt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Train YOLOv8m on custom dataset for 20 epochs with frozen backbone (first 10 layers) to stabilize detection heads
model.train(
    data = DATA,
    epochs = 20,
    imgsz = 640,
    batch = 16,
    device = 0,
    workers = 2,
    patience = 30,
    freeze = 10,
    cos_lr = True
)

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/data/object_detection_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aca8c32dbe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [ ]:
# Continue from best weights
model = YOLO("runs/detect/train/weights/best.pt")

# Fine-tune all layers of YOLOv8 at 960px resolution using best weights for maximum accuracy and convergence stability
model.train(
    data = DATA,
    epochs = 50,
    imgsz = 960,
    batch = 8,
    device = 0,
    workers = 2,
    patience = 30,
    freeze = 0,
    cos_lr = True,
    lr0 = 0.0018,
    close_mosaic=10,
)

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/data/object_detection_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0018, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aca54cba120>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [ ]:
# Evaluate the fine-tuned model on the validation set using 960px images with test-time augmentation (TTA);
# generates precision-recall, confusion matrix, and mAP plots, and prints overall performance metrics.
metrics = model.val(data=DATA, imgsz=960, augment=True, plots=True)
print(metrics)

# Copy the best-trained YOLOv8 weights to Google Drive for permanent storage
!cp runs/detect/train2/weights/best.pt /content/drive/MyDrive/benthic_yolov8_best.pt

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
Model summary (fused): 92 layers, 25,843,813 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 60.5±15.5 MB/s, size: 94.7 KB)
val: Scanning /content/drive/MyDrive/data/object_detection_dataset/val/labels.cache... 551 images, 8 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 551/551 717.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 3.2it/s 10.8s
                   all        551        575      0.843      0.825       0.89      0.622
                  crab         84         89      0.921      0.783      0.899        0.6
                   eel         89         89      0.892      0.921      0.961       0.77
              flatfish         62         62      0.777       0.73      0.848      0.487
             roundfish         78         82      0.828      0.824      0.883      0.586
 

In [ ]:
# Run inference on test images at 960px for maximum accuracy and visual quality
model.predict(
    source="/content/drive/MyDrive/data/object_detection_dataset/test/images",
    imgsz = 960,
    conf = 0.25,
    iou = 0.6,
    augment = True,
    save = True
)


image 1/277 /content/drive/MyDrive/data/object_detection_dataset/test/images/Eel-105_jpg.rf.8b19548cd857fa2fddac62e7039cb2db.jpg: 960x960 2 eels, 160.2ms
image 2/277 /content/drive/MyDrive/data/object_detection_dataset/test/images/Eel-107_jpg.rf.d6b7afebcb8a2f5ca372008139bb7209.jpg: 960x960 1 eel, 28.4ms
image 3/277 /content/drive/MyDrive/data/object_detection_dataset/test/images/Eel-114_jpg.rf.1b27fddc64b559039c0e663f0755b4f0.jpg: 960x960 1 eel, 30.6ms
image 4/277 /content/drive/MyDrive/data/object_detection_dataset/test/images/Eel-123_jpg.rf.4058d600133f505811681ea74602807d.jpg: 960x960 1 eel, 30.5ms
image 5/277 /content/drive/MyDrive/data/object_detection_dataset/test/images/Eel-134_jpg.rf.4d61fa4a11c63aa16b1b30de3e393091.jpg: 960x960 1 eel, 28.6ms
image 6/277 /content/drive/MyDrive/data/object_detection_dataset/test/images/Eel-169_jpg.rf.6565dc2a02f657d887e41a37bc07b760.jpg: 960x960 1 eel, 28.6ms
image 7/277 /content/drive/MyDrive/data/object_detection_dataset/test/images/Eel-186_

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'crab', 1: 'eel', 2: 'flatfish', 3: 'roundfish', 4: 'scallop', 5: 'skate', 6: 'whelk'}
 obb: None
 orig_img: array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [65, 49, 56],
         [45, 27, 40],
         [63, 44, 59]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [63, 51, 57],
         [45, 31, 43],
         [53, 38, 52]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [55, 51, 56],
         [41, 37, 48],
         [44, 39, 54]],
 
        ...,
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [38, 32, 37],
         [59, 52, 59],
         [59, 52, 59]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [38, 31, 34],

In [ ]:
# Backup all YOLO training runs, weights, and results to Google Drive for safekeeping
!cp -r /content/runs /content/drive/MyDrive/